# 5.4 在 PyTorch 中加载和保存模型权重

我们讨论了如何从数值上评估训练进度以及如何从零开始预训练一个大型语言模型（LLM）。
尽管大语言模型和数据集都相对较小，但此练习表明预训练 大语言模型的计算成本很高。
因此，能够保存大语言模型非常重要，这样我们就不必每次想要在新会话中使用它时都重新运行训练。

如图 5.16 中的章节概述所示，我们将在本节中介绍如何保存和加载预训练模型。
然后，在接下来的部分中，我们将从 OpenAI 加载一个功能更强大的预训练 GPT 模型到我们的 GPTModel 实例中。

**图 5.16 在训练和检查模型之后，保存模型通常很有帮助的，这样我们以后可以使用或继续训练它，这是本节的主题，然后我们将在本章的最后一节中从 OpenAI 加载预训练的模型权重**

![fig5.16](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-5-16.jpg?raw=true)

幸运的是，保存一个PyTorch 模型相对简单。
这里推荐的方法是使用 torch.save 函数保存模型的所谓状态字典 state_dict，这是一个将每个层映射到其参数的字典， 代码如下所示：

In [ ]:
torch.save(model.state_dict(), "model.pth")

在上面的代码中，“model.pth”是保存state_dict的文件名。 
从技术上讲我们可以使用任何文件扩展名，但.pth扩展名是PyTorch文件的常规约定

然后，通过 state_dict 保存模型权重后，我们可以按照以下代码将模型权重加载到新的 GPTModel 模型实例中：

In [ ]:
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model.pth"))
model.eval()

正如第 4 章所讨论的，dropout通过在训练期间随机“丢弃”层的神经元来帮助防止模型对训练数据过拟合。
然而，在推理过程中，我们不想随机丢弃网络学到的任何信息。
使用 model.eval() 能够将模型切换到评估模式进行推理，禁用模型的 dropout 层。

如果我们计划稍后继续预训练模型，例如使用本章早些时候定义的train_model_simple函数，建议同时保存优化器的状态。
这样做可以在重新开始训练时保持优化器的所有参数和状态，从而更有效地继续训练过程。

AdamW 等自适应优化器，会为每个模型权重存储附加参数。
AdamW 利用历史数据动态调整每个模型参数的学习率。
如果没有它，优化器将重置，模型可能学习效果不佳，甚至无法正确收敛，这意味着它将失去生成连贯文本的能力。
使用torch.save，我们可以按照以下方式保存模型和优化器state_dict内容：

In [ ]:
torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    },
    "model_and_optimizer.pth"
)

接着，我们可以通过首先使用torch.load加载保存的数据，然后使用load_state_dict方法来恢复模型和优化器的状态，具体操作如下：

In [ ]:
checkpoint = torch.load("model_and_optimizer.pth")
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train();

### 练习5.4

在保存权重后，在一个新的Python会话或Jupyter笔记本文件中加载模型和优化器，并使用train_model_simple函数继续对其进行预训练1个周期。这样做可以无缝地继续之前的训练过程，从而提高模型的性能和适应性。